### imports & data

In [6]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import mlflow
import mlflow.sklearn
import dagshub

In [4]:
df = pd.read_excel(r"I:\CampusX_DS\campusx_dsmp2\9. MLOps revisited\tutorial\Used-Car-Price-Predictor\experiment\cars24_v3.xlsx")

In [5]:
import dagshub
dagshub.init(repo_owner='iamprashantjain', repo_name='Used-Car-Price-Predictor', mlflow=True)
mlflow.set_tracking_uri("https://dagshub.com/iamprashantjain/Used-Car-Price-Predictor.mlflow")
mlflow.set_experiment('BaseLine Model')

Accessing as iamprashantjain

Initialized MLflow to track repo "iamprashantjain/Used-Car-Price-Predictor"

Repository iamprashantjain/Used-Car-Price-Predictor initialized!

2025/07/18 14:08:14 INFO mlflow.tracking.fluent: Experiment with name 'BaseLine Model' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/986edfba90e24830a2050d400151c3c4', creation_time=1752827895205, experiment_id='0', last_update_time=1752827895205, lifecycle_stage='active', name='BaseLine Model', tags={}>

### experiment 1 - find best model

In [7]:
# --- Models ---
regression_models = {
    "RandomForestRegressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor()
}

# --- Preprocessing ---
numerical_cols = ['listingPrice', 'odometer', 'fitnessAge', 'cashDownPayment', 'emiStartingValue',
                  'emiEndingValue', 'roiMinDiscounted', 'roiMaxDiscounted', 'roiMinOriginal',
                  'roiMaxOriginal', 'emiOriginalStartingValue', 'emiOriginalEndingValue', 'featureCount', 'avgEmi']

categorical_cols = ['make', 'model', 'variant', 'year', 'transmissionType', 'bodyType', 'fuelType',
                    'ownership', 'color', '360DegreeCamera', 'AlloyWheels', 'AppleCarplayAndroidAuto',
                    'Bluetooth', 'CruiseControl', 'GpsNavigation', 'InfotainmentSystem', 'LeatherSeats',
                    'ParkingAssist', 'PushButtonStart', 'RearAc', 'SpecialRegNo', 'Sunroof/Moonroof',
                    'TopModel', 'Tpms', 'VentilatedSeats']

# Impute missing values
numerical_imputer = SimpleImputer(strategy='mean')
categorical_imputer = SimpleImputer(strategy='most_frequent')

df[numerical_cols] = numerical_imputer.fit_transform(df[numerical_cols])
df[categorical_cols] = categorical_imputer.fit_transform(df[categorical_cols])

# Encode categorical
encoder = OneHotEncoder(sparse=False, drop='first')
encoded_categorical = encoder.fit_transform(df[categorical_cols])
encoded_df = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out(categorical_cols))

# Combine
df_encoded = pd.concat([df[numerical_cols], encoded_df], axis=1)

# Split X and y
X = df_encoded.drop(columns=["listingPrice"])
y = df_encoded["listingPrice"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- MLflow Tracking ---
with mlflow.start_run(run_name="RandomForest_vs_XGBoost") as parent_run:
    mlflow.set_tag("experiment_type", "regression_comparison_2models")
    mlflow.log_param("test_size", 0.2)

    for model_name, model in regression_models.items():
        with mlflow.start_run(run_name=model_name, nested=True):
            mlflow.log_param("model", model_name)

            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            mae = mean_absolute_error(y_test, y_pred)
            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)

            mlflow.log_metric("mae", mae)
            mlflow.log_metric("mse", mse)
            mlflow.log_metric("r2_score", r2)

            mlflow.sklearn.log_model(model, f"{model_name}_model")

            print(f"\nModel: {model_name}")
            print(f"  MAE: {mae:.2f}, MSE: {mse:.2f}, R2: {r2:.2f}")
            print("-" * 40)


c:\Users\iampr\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
2025/07/18 14:13:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Model: RandomForestRegressor
  MAE: 23474.94, MSE: 19281890090.47, R2: 0.94
----------------------------------------


2025/07/18 14:13:31 INFO mlflow.tracking._tracking_service.client: 🏃 View run RandomForestRegressor at: https://dagshub.com/iamprashantjain/Used-Car-Price-Predictor.mlflow/#/experiments/0/runs/c92e9c112bf245f0a3741dd79ca863f6.
2025/07/18 14:13:31 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/iamprashantjain/Used-Car-Price-Predictor.mlflow/#/experiments/0.
2025/07/18 14:13:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Model: XGBRegressor
  MAE: 34178.28, MSE: 36337214232.42, R2: 0.88
----------------------------------------


2025/07/18 14:14:02 INFO mlflow.tracking._tracking_service.client: 🏃 View run XGBRegressor at: https://dagshub.com/iamprashantjain/Used-Car-Price-Predictor.mlflow/#/experiments/0/runs/070c72fdff4a4d9b93970f0306b324ef.
2025/07/18 14:14:02 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/iamprashantjain/Used-Car-Price-Predictor.mlflow/#/experiments/0.
2025/07/18 14:14:03 INFO mlflow.tracking._tracking_service.client: 🏃 View run RandomForest_vs_XGBoost at: https://dagshub.com/iamprashantjain/Used-Car-Price-Predictor.mlflow/#/experiments/0/runs/aa49f7abea8547b3bd78287c7ac30ceb.
2025/07/18 14:14:03 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/iamprashantjain/Used-Car-Price-Predictor.mlflow/#/experiments/0.


### experiment 2 - find best params

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import mlflow
import mlflow.sklearn
import numpy as np

# --- Define parameter grid ---
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}

# --- MLflow Parent Run ---
with mlflow.start_run(run_name="RandomForest_Hyperparameter_Tuning") as parent_run:
    mlflow.set_tag("experiment_type", "random_forest_hyperparameter_tuning")

    best_r2 = -np.inf
    best_params = None
    best_model = None

    run_counter = 1

    for n_estimators in param_grid['n_estimators']:
        for max_depth in param_grid['max_depth']:
            for min_samples_split in param_grid['min_samples_split']:
                params = {
                    'n_estimators': n_estimators,
                    'max_depth': max_depth,
                    'min_samples_split': min_samples_split
                }

                with mlflow.start_run(run_name=f"run_{run_counter}", nested=True):
                    mlflow.log_params(params)

                    model = RandomForestRegressor(**params, random_state=42)
                    model.fit(X_train, y_train)
                    y_pred = model.predict(X_test)

                    mae = mean_absolute_error(y_test, y_pred)
                    mse = mean_squared_error(y_test, y_pred)
                    r2 = r2_score(y_test, y_pred)

                    mlflow.log_metrics({
                        "mae": mae,
                        "mse": mse,
                        "r2_score": r2
                    })

                    mlflow.sklearn.log_model(model, "model")

                    print(f"Run {run_counter}: R2={r2:.4f}, Params={params}")

                    if r2 > best_r2:
                        best_r2 = r2
                        best_params = params
                        best_model = model

                    run_counter += 1

    # Log best model in parent run
    mlflow.log_params({f"best_{k}": v for k, v in best_params.items()})
    mlflow.log_metric("best_r2_score", best_r2)
    mlflow.sklearn.log_model(best_model, "best_random_forest_model")

    print(f"\n✅ Best Model R2: {best_r2:.4f}, Best Params: {best_params}")

2025/07/18 14:17:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run 1: R2=0.9348, Params={'n_estimators': 100, 'max_depth': None, 'min_samples_split': 2}


2025/07/18 14:18:08 INFO mlflow.tracking._tracking_service.client: 🏃 View run run_1 at: https://dagshub.com/iamprashantjain/Used-Car-Price-Predictor.mlflow/#/experiments/0/runs/72be8ecafd884df0943b06cef31c5840.
2025/07/18 14:18:08 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/iamprashantjain/Used-Car-Price-Predictor.mlflow/#/experiments/0.
2025/07/18 14:19:40 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run 2: R2=0.9368, Params={'n_estimators': 100, 'max_depth': None, 'min_samples_split': 5}


2025/07/18 14:19:48 INFO mlflow.tracking._tracking_service.client: 🏃 View run run_2 at: https://dagshub.com/iamprashantjain/Used-Car-Price-Predictor.mlflow/#/experiments/0/runs/81f26bc8e86d40eaa2bfcec4b750a914.
2025/07/18 14:19:48 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/iamprashantjain/Used-Car-Price-Predictor.mlflow/#/experiments/0.
2025/07/18 14:21:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

2025/07/18 15:01:22 INFO mlflow.tracking._tracking_service.client: 🏃 View run run_17 at: https://dagshub.com/iamprashantjain/Used-Car-Price-Predictor.mlflow/#/experiments/0/runs/b01af7de18a6415cb9d1ba0028bb7082.
2025/07/18 15:01:22 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/iamprashantjain/Used-Car-Price-Predictor.mlflow/#/experiments/0.
2025/07/18 15:04:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run 18: R2=0.9270, Params={'n_estimators': 300, 'max_depth': 20, 'min_samples_split': 5}


2025/07/18 15:04:38 INFO mlflow.tracking._tracking_service.client: 🏃 View run run_18 at: https://dagshub.com/iamprashantjain/Used-Car-Price-Predictor.mlflow/#/experiments/0/runs/53a3a093b2114319b380f659e38e5536.
2025/07/18 15:04:38 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/iamprashantjain/Used-Car-Price-Predictor.mlflow/#/experiments/0.
2025/07/18 15:04:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



✅ Best Model R2: 0.9407, Best Params: {'n_estimators': 100, 'max_depth': 10, 'min_samples_split': 2}


2025/07/18 15:04:50 INFO mlflow.tracking._tracking_service.client: 🏃 View run RandomForest_Hyperparameter_Tuning at: https://dagshub.com/iamprashantjain/Used-Car-Price-Predictor.mlflow/#/experiments/0/runs/446ada565df54c9bb616a4ebd285d335.
2025/07/18 15:04:50 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/iamprashantjain/Used-Car-Price-Predictor.mlflow/#/experiments/0.
